# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902042


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:36,  3.45s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<01:00,  2.43s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:39,  1.73s/it]

Rendering models:  33%|███▎      | 10/30 [00:07<00:24,  1.23s/it]

...

Rendering models:  80%|████████  | 24/30 [00:08<00:01,  5.38it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  6.36it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  6.76it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  7.36it/s]

equidad1                              0.079377
kayleebug2017                         0.002732
Lukes0404                             0.001573
jvillavargas                          0.000245
not-logged-in-134238199d806cd7ef54    0.001577
Drew_Farnsworth                       0.000251
Thedunkmasta                          0.003417
not-logged-in-7526601586eaa938b3ee    0.001860
mlang2017                             0.113082
Synapsestar                           0.000353
relseidy                              0.002628
crush202020                           0.000264
UnbiasedBrigade                       0.000915
not-logged-in-d04ed8c60cd5453cb408    0.000229
peakscience7                          0.018091
not-logged-in-214814e52004b42491a4    0.778771
acapirala                             0.001114
Tyler_Arirama                         0.001012
Lavadude                              0.015544
Mr_DMan                               0.001459
not-logged-in-7cdc0c77b24d18e21c70    0.008541
helios1235   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())